In [1]:
import numpy as np
import argparse
import c3.libraries.fidelities as fidelities
from utils import *
import c3.libraries.algorithms as algorithms
from SingleQubitExperiment import SingleQubitExperiment

2021-09-13 09:48:41.200841: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-13 09:48:41.201105: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
parser = argparse.ArgumentParser()
parser.add_argument("output", help="Output directory")
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] output
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

/home/user/c3/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:

def test_fidelity(propagators: dict, instructions: dict, index, dims,
                  qubit: chip.Qubit, generator: Generator, drive: chip.Drive, n_eval=-1):
    numPeaks = 2
    # find energies of the qubit
    energies = qubit.get_Hamiltonian().numpy().diagonal().real
    vec1 = np.array([energies[3] - energies[2], energies[1] - energies[0]]) / (2 * np.pi)

    infids = []
    for gate, propagator in propagators.items():
        # create signal and find peaks
        instr = instructions[gate]
        signal = generator.generate_signals(instr)[drive.name]
        peakFrequencies, peakValues = findFrequencyPeaks(signal["ts"].numpy(), signal["values"].numpy(), numPeaks)

        # add frequency offsets to infidelity
        #infid = np.linalg.norm(vec1 - peakFrequencies)
        infid1 = np.abs(vec1[0] - peakFrequencies[0]) / peakFrequencies[0]
        infid1 += np.abs(vec1[1] - peakFrequencies[1]) / peakFrequencies[1]

        # add peak height difference to infidelity
        if peakFrequencies[1] > peakFrequencies[0]:
            a = peakValues[1]
            b = np.sqrt(2) * peakValues[0]
        else:
            a = peakValues[0]
            b = np.sqrt(2) * peakValues[1]
        infid2 = np.abs(a - b) / np.maximum(a, b)
        infid = infid1 * (1 + infid2)
        infids.append(infid)
    return tf.reduce_mean(infids)


def printInformation(exper: SingleQubitExperiment, name: str) -> None:
    signal = exper.generateSignal()
    peakFrequencies, peakValues = findFrequencyPeaks(signal["ts"].numpy(), signal["values"].numpy(), 4)
    print("peaks: ", np.sort(peakFrequencies))
    exper.plotSignal("signal_" + name)
    exper.plotTimeEvolution("populations_" + name)
    exper.plotPropagator("propagator_" + name)


def setOptimalValues(envelope: pulse.Envelope, params: dict) -> None:
    for key,value in params.items():
        envelope.params[key].set_value(value)

In [7]:
'''
optimal_values = {
    "amp": 0.5982053163520968,
    #"sigma": 0.020022993947430123,
    "xy_angle": -1.855052604814916,
    "freq_offset": -54371228.73199173,
    "delta": -0.14784432135434766,
    #"carrier-freq": 5222459053.481677,
    #"framechange": 2.5942825630048745
}
inphase = tf.convert_to_tensor([0.2860119617309102, 0.2515755091876439, 0.173873025201375, 0.0007877386279906926, 0.013990268564515115, 1.0829353284431187, 0.12363579320739716, 0.533448750207164, 0.8241718971826906, 1.0126588320674026, 0.14054899938732912, 0.4154462095036047, 0.27790851911436115, 0.23695097592812822, 0.77387916183715, 0.13882356089751952, 0.17031455162574796, 0.015579040202510648, 0.10100116240706104, 0.8967235745253517, 0.2340774787995677, 0.4065161651870501, 0.14357762493498727, 0.9778244879245249, 0.21827392144642413, 0.7419575443378293, 0.9844786602582266, 0.774177092939803, 1.034049069101188, 0.8755064255501308, 0.6625810857937209, 0.5166892426294151, 1.0869475685574725, 1.0267639229288643, 0.9169247779955447, 0.05928266478720349, 0.375552034589335, 0.1452760805899441, 0.8011251527567873, 0.1722525872387966, 0.6124577751630437, 0.15228718221912907, 0.013159047422195423, 0.2084289367907402, 0.965936486818863, 0.4794322168470483, 7.777490527675845e-05, 1.0527621402872498, 0.5854932409328775, 0.8591261219539706, 0.27456047754898577, 0.03254927163527047, 0.8025139099221452, 1.0105039386343857, 0.9750845205896466, 0.06200624813674867, 0.3584582169425935, 0.4312511943969614, 0.6545680324609726, 0.18879975173915983])
'''

freq = 5e9 #5222459053.481677
anharm = 330e6
t_final = 20e-9
sigma = t_final / 5 #0.020022993947430123
sigma2 = sigma / 2
relative_amp = 2
carrier_freq = 4.7e9
optimisable_params = {
    "gauss": ["amp", #"sigma",
              "xy_angle", "freq_offset",  # for single gaussian envelope
              #"t_final",
              #"t_bin_start", "t_bin_end",
              #"sigma2", "relative_amp",  # for double-gaussian envelope
              #"delta",  # for DRAG
              "inphase",
              ],
    "carrier": [
        "freq",
        "framechange"
    ]
}

In [8]:
#for i, relative_amp in enumerate(np.linspace(0.5, 4, 50)):
for i in range(5):
    envelope = createPWCDoubleGaussianPulse(t_final, sigma, sigma2, relative_amp, 30)
    #setOptimalValues(envelope, optimal_values)
    exper = SingleQubitExperiment(directory=f"{args.output}/", file_suffix=f"{i}")
    exper.prepare(t_final, freq, anharm, carrier_freq, envelope, useDrag=False)

    # print information
    energies = exper.getEnergies()
    print("energies: ", [(energies[i + 1] - energies[i]) / (2 * np.pi) for i in range(len(energies) - 1)])
    printInformation(exper, "before")

    #params_mid = exper.optimise(
    #    optimisable_params,
    #    algorithm=algorithms.cmaes,
    #    algorithm_options={
    #        "stop_at_convergence": 15,
    #        "spread": 0.2,
    #        "popsize": 25,
    #    },
    #    fidelity_function=test_fidelity,
    #    fidelity_params={
    #        "qubit": exper.getQubit(),
    #        "generator": exper.getExperiment().pmap.generator,
    #        "drive": getDrive(exper.getExperiment().pmap.model, exper.getQubit())
    #    },
    #)
    #printInformation(exper, "mid")
    #print(params_mid)

    # second optimisation
    params_after = exper.optimise(
        optimisable_params,
        algorithm=algorithms.cmaes,
        algorithm_options={
            "stop_at_convergence": 20,
            "spread": 0.2,
            "popsize": 25,
        },
        fidelity_function=fidelities.unitary_infid_set,
        fidelity_params={
            "active_levels": 4,
        },
    )
    printInformation(exper, "after")
    print(params_after)

NameError: name 'args' is not defined

In [4]:
'''
times = np.arange(1e-9, 20e-9, 5e-12)[501:1000]
phases = np.zeros((4, len(times)), dtype=np.complex128)
print(phases.shape)
for idx,t_final in enumerate(times):
    print(idx, " / ", len(phases[0]))
    envelope = createNoDrivePulse(t_final)
    exper = SingleQubitExperiment(directory=None)
    exper.prepare(t_final, freq, anharm, carrier_freq, envelope, useDrag=False)
    propagator = exper.getExperiment().propagators["lower-X[0]"]
    state = tf.matmul(propagator, exper.getInitialState()).numpy()
    for i in range(4):
        phases[i, idx] = state[i]
    del state
    del propagator
    del exper
    del envelope
np.save('./phases_1.npy', phases)
'''

(4, 499)
0  /  499


2021-09-09 11:49:19.848242: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-09 11:49:19.848636: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-09 11:49:19.848673: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-09 11:49:19.848729: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (localhost.localdomain): /proc/driver/nvidia/version does not exist
2021-09-09 11:49:19.849594: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-09 11:49:20.695865: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-09 11:4

1  /  499
2  /  499
3  /  499
4  /  499
5  /  499
6  /  499
7  /  499
8  /  499
9  /  499
10  /  499
11  /  499
12  /  499
13  /  499
14  /  499
15  /  499
16  /  499
17  /  499
18  /  499
19  /  499
20  /  499
21  /  499
22  /  499
23  /  499
24  /  499
25  /  499
26  /  499
27  /  499
28  /  499
29  /  499
30  /  499
31  /  499
32  /  499
33  /  499
34  /  499
35  /  499
36  /  499
37  /  499
38  /  499
39  /  499
40  /  499
41  /  499
42  /  499
43  /  499
44  /  499
45  /  499
46  /  499
47  /  499
48  /  499
49  /  499
50  /  499
51  /  499
52  /  499
53  /  499
54  /  499
55  /  499
56  /  499
57  /  499
58  /  499
59  /  499
60  /  499
61  /  499
62  /  499
63  /  499
64  /  499
65  /  499
66  /  499
67  /  499
68  /  499
69  /  499
70  /  499
71  /  499
72  /  499
73  /  499
74  /  499
75  /  499
76  /  499
77  /  499
78  /  499
79  /  499
80  /  499
81  /  499
82  /  499
83  /  499
84  /  499
85  /  499
86  /  499
87  /  499
88  /  499
89  /  499
90  /  499
91  /  499
92  /  4